# Benchmark the solver

## Package versions

In [1]:
import gurobipy as gb
from gurobipy import GRB

import scipy.sparse as sp
from scipy.optimize import linprog, milp, Bounds, LinearConstraint
import scipy

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
print("numpy version: ", np.__version__)
print("pandas version: ", pd.__version__)
print("scipy version: ", scipy.__version__)
# print("gekko version: ", m.version)

cvxpy version:  1.2.1
pyomo version:  6.4.2
numpy version:  1.23.3
pandas version:  1.5.0


## Benchmark case

min. $ c^Tx + d $

s.t.

$ Gx \leq h $

$ Ax = b $

### Data generation

In [3]:
def data_prep(x_len, G_len, A_len):
    np.random.seed(2022)
    c = np.random.randn(x_len)

    np.random.seed(2021)
    d = np.random.randn(x_len)

    np.random.seed(2020)
    G = np.random.randn(G_len, x_len)

    np.random.seed(2019)
    h = np.random.randn(G_len)

    np.random.seed(2018)
    A = np.random.randn(A_len, x_len)

    np.random.seed(2017)
    b = np.random.randn(A_len)
    return c, d, G, h, A, b

x_len = 2000
G_len = 4000
A_len = 2000
c, d, G, h, A, b = data_prep(2*x_len, G_len, A_len)

In [4]:
def gbm1(c, d, G, h, A, b, x_len):
    # Create a new model
    m1 = gb.Model("m1")
    # Create variables
    x = m1.addMVar(shape=2*x_len, vtype=GRB.BINARY, name="x")
    x[0:x_len].setAttr("vtype", GRB.CONTINUOUS)
    m1.setObjective(c @ x, GRB.MINIMIZE)

    # Add constraints
    m1.addConstr(G @ x <= h, name="ic")
    m1.addConstr(A @ x == b, name="ec")
    m1.update()
    return m1

def gbm2(c, d, G, h, A, b, x_len):
    # Create a new model
    m2 = gb.Model("m2")
    x1 = m2.addVars(range(x_len), vtype=GRB.CONTINUOUS, name='x1')
    x2 = m2.addVars(range(x_len), vtype=GRB.BINARY, name='x2')
    obj1 = sum(x1[gen] * c[gen] for gen in range(x_len))
    obj2 = sum(x2[gen] * c[gen + x_len] for gen in range(x_len))
    m2.setObjective(expr=obj1 + obj2, sense=gb.GRB.MINIMIZE)

    # Add constraints
    ec1 = sum(A[0, gen] * x1[gen] for gen in range(x_len))
    ec2 = sum(A[1, gen] * x2[gen] for gen in range(x_len))

    for i in range(A_len):
        ec = sum(A[i, gen] * x1[gen] for gen in range(x_len))
        m2.addConstr(ec == b[i], name=f'ec{i}')

    for i in range(G_len):
        ic = sum(G[i, gen] * x1[gen] for gen in range(x_len))
        m2.addConstr(ic == h[i], name=f'ic{i}')
    m2.update()
    return m2

In [5]:
gbm1(c, d, G, h, A, b, x_len)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-04-08


<gurobi.Model MIP instance m1: 6000 constrs, 4000 vars, No parameter changes>

In [6]:
%%timeit  -r5 -n5
gbm1(c, d, G, h, A, b, x_len)

1.25 s ± 17 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)


In [ ]:
%%timeit  -r5 -n5
gbm2(c, d, G, h, A, b, x_len)

### Solve with gurobi

In [ ]:
## gurobi

In [ ]:
# # Generate a random problem
# np.random.seed(0)
# m, n= 40, 25

# A = np.random.rand(m, n)
# b = np.random.randn(m)

# # Construct a CVXPY problem
# x = cp.Variable(n, integer=True)
# objective = cp.Minimize(cp.sum_squares(A @ x - b))
# prob = cp.Problem(objective)
# prob.solve(solver='CVXOPT')